<a href="https://colab.research.google.com/github/1900690/kyouyu/blob/main/Matlab_in_Google_Colab.0.0.1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#The command below takes about 2 minutes to execute.

In [1]:
!apt install octave

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-510
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  aglfn epstool fonts-droid-fallback fonts-freefont-otf fonts-noto-mono
  fonts-urw-base35 ghostscript gnuplot-data gnuplot-qt gsfonts
  imagemagick-6-common info install-info libamd2 libauthen-sasl-perl libcamd2
  libccolamd2 libcholmod3 libcolamd2 libcxsparse3 libdata-dump-perl libemf1
  libencode-locale-perl libfftw3-single3 libfile-listing-perl libfltk-gl1.3
  libfltk1.3 libfont-afm-perl libglpk40 libgraphicsmagick++-q16-12
  libgraphicsmagick-q16-3 libgs9 libgs9-common libhtml-form-perl
  libhtml-format-perl libhtml-parser-perl libhtml-tagset-perl
  libhtml-tree-perl libhttp-cookies-perl libhttp-daemon-perl libhttp-date-perl
  libhttp-message-perl libhttp-negotiate-perl libidn11 libijs-0.35
  libio-htm

#Simple Example of Matlab code in Colab

In [2]:
%%writefile add.m
x = 3
y = 2
display(x+y)

Writing add.m


In [4]:
# -W : no window system
!octave -W add.m

octave: X11 DISPLAY environment variable not set
octave: disabling GUI features
x =  3
y =  2
 5


Black Scholes Model and results are correct

In [5]:
%%writefile bs.m

S = 100
X = 100
r = 0.05
v = 0.2
T = 1

%Black and Scholes in Matlab

d1=(log(S/X)+(r+v^2/2)*T)/(v*sqrt(T));
d2=d1-v*sqrt(T);
c = S*normcdf(d1)-X*exp(-r*T)*normcdf(d2)




Writing bs.m


In [6]:
# -W : no window system
!octave -W bs.m

S =  100
X =  100
r =  0.050000
v =  0.20000
T =  1
Forge which seems to not be installed in your system.

Please read <https://www.octave.org/missing.html> to learn how you can
contribute missing functionality.
error: 'normcdf' undefined near line 12 column 7
error: called from
    bs at line 12 column 3


#Results below have not been checked for Trinomial Model. Chen's code is correct but I have not verified my own Implementation - so please treat with great care and please do not quote me on results produced below.

https://uk.mathworks.com/matlabcentral/fileexchange/50177-lattice-methods-for-option-pricing

Boyle Trinomial

In [7]:
%%writefile triBoyle.m

%function [OptionPrice, TrinLatticeUnderlying, TrinLatticeOption] = ...
%                   priceOptionTrinBoyle(Underlying, Strike, Rate, T, ...
%                  NumOfTimeSteps, Sigma, StrCallPut)
% The function calculates a standard European option price by using
% the Boyle trinomial model
% Ref: Kamrad, B., Ritchken, P., 1991. 
% Multinomial approximating models for options with k state variables. 
% Management Science 37 (23), 1640¨C1652.
%
% Input: 
% Underlying: initial value of underlying price
% Strike: strike/exercise price
% Rate: continuous-time risk-less rate
% T: in terms of year format
% NumOfTimeSteps: how many steps specified for T, where dt := T/TimeSteps
% Sigma: volatility
% StrCallPut: 'Call' or 'Put' (default is 'Call')
%
% Output:
% OptionPrice
% BinLatticeUnderlying
% BinLatticeOption
% 
% Bowei Chen | PhD
% University College London
% bowei.chen@cs.ucl.ac.uk
% 03/2015   

Underlying = 100
Strike = 100
Rate = 0.05
T = 1
NumOfTimeSteps = 100
Sigma = 0.2
StrCallPut = 'Call' 

function PayoffValue = calcPayoff(Underlying, Strike, StrCallPut)
% The function calculates a standard European option payoff 

if strcmp(StrCallPut, 'put') || strcmp(StrCallPut, 'Put')
    PayoffValue = max(Strike - Underlying, 0);
else
    PayoffValue = max(Underlying - Strike, 0);
end
end % end of func

function DataStructure = buildLattice(NumOfTimeSteps, StrBinTrin)
% The function creates a binomial/trinomial lattice 
% Input: 
% NumOfTimeSteps
% StrBinTrin
%
% Output:
% LatticeStructure (start with time 1)

DataStructure = '';
if strcmp(StrBinTrin, 'trin') || strcmp(StrBinTrin, 'Trin')
    for i = 1:NumOfTimeSteps    
        DataStructure{i} = nan(1,2*i-1);    
    end  
else
    for i = 1:NumOfTimeSteps    
        DataStructure{i} = nan(1,i);      
    end
end
end % end of func

%% Initialization
dt = T / NumOfTimeSteps;
r = exp(Rate * dt); 
lam = 1.1;
g = r^2 * (exp(Sigma^2 * dt) - 1);
u = exp(lam * Sigma * sqrt(dt));
m = 1;
d = exp(- lam * Sigma * sqrt(dt));
q1 = ((g + r^2 - r)*u - (r-1))/((u-1)*(u^2-1));
q3 = ((g + r^2 - r)*u^2 - (r-1) * u^3)/((u-1)*(u^2-1));
q2 = 1 - q1 - q3;
%% Loop over each node of underlying price tree
TrinLatticeUnderlying = buildLattice(NumOfTimeSteps+1, 'trin');
TrinLatticeUnderlying{1}(1) = Underlying;
for Idx = 2:NumOfTimeSteps+1
    TrinLatticeUnderlying{Idx}(1) = TrinLatticeUnderlying{Idx-1}(1)*u;
    TrinLatticeUnderlying{Idx}(2) = TrinLatticeUnderlying{Idx-1}(1)*m; 
    TrinLatticeUnderlying{Idx}(3) = TrinLatticeUnderlying{Idx-1}(1)*d;  
    x = length(TrinLatticeUnderlying{Idx});
    if  x > 3
        for k = 4:x
            TrinLatticeUnderlying{Idx}(k) = TrinLatticeUnderlying{Idx-1}(k-2)*d;
        end
    end
    clear x
end
%% Calculate the value at expiry
TrinLatticeOption = buildLattice(NumOfTimeSteps+1, 'trin');
for k = 1:length(TrinLatticeOption{end})
    TrinLatticeOption{end}(k) = calcPayoff(TrinLatticeUnderlying{end}(k), ...
        Strike, StrCallPut);
end
%% Loop backwards to get values at the earlier times
for Idx = NumOfTimeSteps:-1:1
    for k = 1:length(TrinLatticeOption{Idx})
        TrinLatticeOption{Idx}(k) = ...
            r^-1 * (q1 * TrinLatticeOption{Idx+1}(k) ...
                + q2 * TrinLatticeOption{Idx+1}(k+1) ... 
                + q3 * TrinLatticeOption{Idx+1}(k+2));
    end
end
OptionPrice = TrinLatticeOption{1}(1); 

OptionPrice


Writing triBoyle.m


In [8]:
# -W : no window system
!octave -W triBoyle.m

Underlying =  100
Strike =  100
Rate =  0.050000
T =  1
NumOfTimeSteps =  100
Sigma =  0.20000
StrCallPut = Call
OptionPrice =  10.452


https://uk.mathworks.com/matlabcentral/fileexchange/50177-lattice-methods-for-option-pricing

In [9]:
%%writefile triKR.m

Underlying = 100
Strike = 100
Rate = 0.05
T = 1
NumOfTimeSteps = 100
Sigma = 0.2
StrCallPut = 'Call' 

function PayoffValue = calcPayoff(Underlying, Strike, StrCallPut)
% The function calculates a standard European option payoff 

if strcmp(StrCallPut, 'put') || strcmp(StrCallPut, 'Put')
    PayoffValue = max(Strike - Underlying, 0);
else
    PayoffValue = max(Underlying - Strike, 0);
end
end % end of func

function DataStructure = buildLattice(NumOfTimeSteps, StrBinTrin)
% The function creates a binomial/trinomial lattice 
% Input: 
% NumOfTimeSteps
% StrBinTrin
%
% Output:
% LatticeStructure (start with time 1)

DataStructure = '';
if strcmp(StrBinTrin, 'trin') || strcmp(StrBinTrin, 'Trin')
    for i = 1:NumOfTimeSteps    
        DataStructure{i} = nan(1,2*i-1);    
    end  
else
    for i = 1:NumOfTimeSteps    
        DataStructure{i} = nan(1,i);      
    end
end
end % end of func

%% Initialization
%% Initialization
dt = T / NumOfTimeSteps;
r = exp(Rate * dt); 
lam = 1.3;
u = exp(lam * Sigma * sqrt(dt));
m = 1;
d = exp(- lam * Sigma * sqrt(dt));
q1 = 1/(2*lam^2) + ((Rate - 1/2 * Sigma^2) * sqrt(dt))/(2 * lam * Sigma);
q2 = 1 - 1/(lam^2);
q3 = 1/(2*lam^2) - ((Rate - 1/2 * Sigma^2) * sqrt(dt))/(2 * lam * Sigma);
%% Loop over each node of underlying price tree
TrinLatticeUnderlying = buildLattice(NumOfTimeSteps+1, 'trin');
TrinLatticeUnderlying{1}(1) = Underlying;
for Idx = 2:NumOfTimeSteps+1
    TrinLatticeUnderlying{Idx}(1) = TrinLatticeUnderlying{Idx-1}(1)*u;
    TrinLatticeUnderlying{Idx}(2) = TrinLatticeUnderlying{Idx-1}(1)*m; 
    TrinLatticeUnderlying{Idx}(3) = TrinLatticeUnderlying{Idx-1}(1)*d;  
    x = length(TrinLatticeUnderlying{Idx});
    if  x > 3
        for k = 4:x
            TrinLatticeUnderlying{Idx}(k) = TrinLatticeUnderlying{Idx-1}(k-2)*d;
        end
    end
    clear x
end
%% Calculate the value at expiry
TrinLatticeOption = buildLattice(NumOfTimeSteps+1, 'trin');
for k = 1:length(TrinLatticeOption{end})
    TrinLatticeOption{end}(k) = calcPayoff(TrinLatticeUnderlying{end}(k), ...
        Strike, StrCallPut);
end
%% Loop backwards to get values at the earlier times
for Idx = NumOfTimeSteps:-1:1
    for k = 1:length(TrinLatticeOption{Idx})
        TrinLatticeOption{Idx}(k) = ...
            r^-1 * (q1 * TrinLatticeOption{Idx+1}(k) ...
                + q2 * TrinLatticeOption{Idx+1}(k+1) ... 
                + q3 * TrinLatticeOption{Idx+1}(k+2));
    end
end
OptionPrice = TrinLatticeOption{1}(1);

OptionPrice

Writing triKR.m


In [10]:
# -W : no window system
!octave -W triKR.m

Underlying =  100
Strike =  100
Rate =  0.050000
T =  1
NumOfTimeSteps =  100
Sigma =  0.20000
StrCallPut = Call
OptionPrice =  10.443
